In [16]:
sc

org.apache.spark.SparkContext@750e2d33

In [17]:
val in = scala.io.Source.fromURL("http://172.31.2.11:31410/db/charmander/series?u=root&p=root&q="+java.net.URLEncoder.encode(" select memory_usage from stats limit 200"), "utf-8")

In [18]:
var data = ""
for (line <- in.getLines)
data = line

In [19]:
import org.json4s.jackson.JsonMethods


In [20]:
val json = JsonMethods.parse(data)

In [21]:
val points = json \\ "points"

In [22]:
val mypoints = points.values
mypoints

List(List(1425193047000, 36000001, 84103168), List(1425193047000, 35400001, 84103168), List(1425193047000, 34800001, 84103168), List(1425193046000, 35990001, 84103168), List(1425193046000, 35390001, 84103168), List(1425193046000, 34790001, 84103168), List(1425193045000, 35980001, 84103168), List(1425193045000, 35380001, 84103168), List(1425193045000, 34780001, 84103168), List(1425193044000, 35970001, 84103168), List(1425193044000, 35370001, 84103168), List(1425193044000, 34770001, 84103168), List(1425193043000, 35960001, 84103168), List(1425193043000, 35360001, 84103168), List(1425193043000, 34760001, 84103168), List(1425193042000, 35950001, 84103168), List(1425193042000, 35350001, 84103168), List(1425193042000, 34750001, 84103168), List(1425193041000, 35940001, 84...

In [23]:
val rdd = sc.parallelize(mypoints.asInstanceOf[List[List[BigDecimal]]])

In [24]:
case class MemoryUsage(timestamp: BigDecimal, memory: BigDecimal)

In [25]:
val memoryusage = rdd.map(p => MemoryUsage(BigDecimal(p(0).asInstanceOf[BigInt]), BigDecimal(p(2).asInstanceOf[BigInt])))

In [26]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext._

In [27]:
memoryusage.registerTempTable("memoryusage")

In [28]:
sqlContext.sql("select max(memory) from memoryusage").first()

[84103168]